<a target="_blank" href="https://colab.research.google.com/github/ArtificialIntelligenceToolkit/aitk/blob/master/notebooks/Coverage.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install aitk --upgrade --quiet

In [2]:
import aitk.robots
from aitk.utils import Grid
aitk.robots.__version__

'0.9.32'

This program allows the robot to wander around an empty world.
It uses a Grid imposed on the World to track the robot's location
over time. It computes 
the percentage of Grid locations visited.

In [3]:
world = aitk.robots.World(width=200, height=200, scale=5.0)
robot = aitk.robots.Scribbler(x=100, y=100, a=87, max_trace_length=60)
robot.add_device(aitk.robots.RangeSensor(position=(6,-6),max=20,a=0,width=57.3,name="left-ir"))
robot.add_device(aitk.robots.RangeSensor(position=(6,6),max=20,a=0,width=57.3,name="right-ir"))
world.add_robot(robot)
world.update()
world.save()

Random seed set to: 470787


In [4]:
world.watch()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

In [5]:
from random import random

def wander(robot):
    left = robot[0].get_distance()
    right = robot[1].get_distance()
    x, y, direction = robot.get_pose()
    robot.state["grid"].update(x, y)
    robot.state["grid"].draw()
    # Save the robot's location only once per second
    if left == robot[0].get_max() and right == robot[1].get_max() and \
        robot.state["timer"] == 0:
        #front clear, move random direction
        direction = 1
        if random() < 0.5:
            direction = -1
        robot.move(0.5, direction*random())
        if robot.state["debug"]: robot.speak("F")
    elif robot.state["timer"] > 0 and robot.state["timer"] < 5:
        #timer triggered, continue current rotation
        robot.state["timer"] += 1
        if robot.state["debug"]: robot.speak("timer %d" % (robot.state["timer"]))
    elif left < robot[0].get_max():
        #obstacle on left, turn right, trigger timer
        robot.move(0, -0.4)
        robot.state["timer"] = 1
        if robot.state["debug"]: robot.speak("R")
    elif right < robot[1].get_max():
        #obstacle on right, turn left, trigger timer
        robot.move(0, 0.4)
        robot.state["timer"] = 1
        if robot.state["debug"]: robot.speak("L")
    else:
        #reset timer to zero
        robot.state["timer"] = 0
        if robot.state["debug"]: robot.speak("reset")    
    

In [14]:
from random import randrange

world.reset()
robot.state["timer"] = 0
robot.state["grid"] = Grid((10,10), world)
robot.state["debug"] = True

world.set_seed(randrange(1,100)) # allow randomness
world.seconds(60, [wander], real_time=False)

Using random seed: 18
Using random seed: 24
Simulation stopped at: 00:01:00.0; speed 249.65 x real time


In [15]:
g = robot.state["grid"]
g.show()
print(f"Visited {int(g.analyze_visits()*100)} percent of the world")

  0   0   0   0   0   0   0   0   0   0 
  0   0   0   0  52  60   0   0   0   0 
  0   0   0   8  34  38   2   0   0   0 
  0   0   0  21  17  53   0   0   0   0 
  0   0   0   0  27  62   0   0   0   0 
  0   0   0   0   0  22   0   0   0   0 
  0   0   0  28  24   5   0   0   0   0 
  0   0  31  70   0   0   0   0   0   0 
  0   0   0  46   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0 

Visited 18 percent of the world
